# PRVI ZADATAK - Viterbijev algoritam u log-spaceu

Izradite Viterbijev algoritam (za povremeno nepoštenu kockarnicu) u log-space-u. Drugim riječima, umjesto s produktima (vjerojatnosti), baratat ćemo sa sumama (logaritama vjerojatnosti).
Kod testirajte na nekom dovoljno dugačkom nizu (barem 1000 znakova).
Parametre modela uzmite iz prethodnog zadatka o kockarnici.

### Običan Viterbi - Bioinformatika 1

In [1]:
x = '123455666666123452'

#retci emisijske matrice
F = [ 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0 ] #FAIR
L = [ 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/2.0] #LOADED

#elementi tranzicijske matrice
cl = 0.9 
dl = 0.1
cf = 0.8
df = 0.2

tmp = []
V = []
for i in range(len(x)):
    tmp.append(0) #array koji sadrži nule, iste duljine kao i x

for i in range(2):
    V.append(tmp[:]) #prazna matrica 2 puta duljina od x


k = int(x[0]) - 1 #wtf

V[0][0] = F[k]
V[1][0] = L[k]


#rekurzivno računamo V
for i in range( 1,  len( x ) ):
    tmp = []
    k = int(x[i]) - 1
    tmp.append(V[0][i-1]*F[k]*cl)
    tmp.append(V[1][i-1]*F[k]*df)
    tmp.sort()
    V[0][i] = tmp[1]
    tmp = []
    tmp.append(V[1][i-1]*L[k]*cf)
    tmp.append(V[0][i-1]*L[k]*dl)
    tmp.sort()
    V[1][i] = tmp[1]    



if V[0][len(V[0])-1] > V[1][len(V[1])-1]:
    l = 0
else:
    l = 1


le = len(V[0]) - 1
put = '' 

while le > 0 or le == 0:
    k = int(x[le])-1
    if le == 0:
        if l == 1:
            put = 'L' + put  
            le = le - 1
        elif l == 0:
            put = 'F' + put 
            le = le - 1
    elif l == 0:
        if (( V[l][le]/( V[l+1][le-1]*(F[k]*df) ) ) > 0.99999999999 and ( V[l][le]/( V[l+1][le-1]*(F[k]*df) ) )<=1.0000000000000003 ):
            put = 'F' + put 
            le = le - 1
            l = l + 1
        elif ( ( V[l][le]/( V[l][le-1]*(F[k]*cl) ) ) > 0.9999999999 and ( V[l][le]/( V[l][le-1]*(F[k]*cl) ) )<=1.0000000000000003 ):
            put = 'F' + put
            le = le - 1
    elif l == 1:
        if ( ( V[l][le]/( V[l-1][le-1]*(L[k]*dl) ) ) > 0.99999999999 and ( V[l][le]/( V[l-1][le-1]*(L[k]*dl) ) )<=1.0000000000000003 ):
            put = 'L' + put  
            le = le - 1
            l = l - 1
        elif ( ( V[l][le]/( V[l][le-1]*(L[k]*cf) ) ) > 0.999999999 and ( V[l][le]/( V[l][le-1]*(L[k]*cf) ) )<=1.0000000000000003 ):
            put = 'L' + put
            le = le - 1        
print(put)   

FFFFFFLLLLLLFFFFFF


### Log-space

Generiramo random uzorak od 1000 brojeva.
Ne uzimamo uniformnu distribuciju jer će nam tada ispasti da se uvijek nalazimo u _fair_ kockici, pa simuliramo uzorak s _wighted_ parametrima (sami odabiremo parametre).

In [22]:
import random
randomlist = random.choices(['1','2','3','4','5','6'], [0.1,0.1,0.1,0.1,0.1,0.3], k=1000)
new_x = "".join([str(int) for int in randomlist])
new_x

'425413163663465566312153156564661132535545646366523163662212121161536161636552426666266465236331665433316366222661662111616363625556466265315152543136361566263326441142621366666264666315124345346616426564246543415356466261533643253244656234364664156654165335416656616225636461516666322561666425366161565616664266262366551413614616163544641416146266561443633535263341616625366214666632526664665564413662542624113524326542661166414113652464315366616656126445326116362462134666636226136661616416233635262313661622631635366354543664616226642646316136641262333553232326412636146433535643632566362224514635462246151652466626461666666566236523622564665215366616163341265645314666451462136536146164336666166626622261562615533343656441461666664666664656466166631446361644244416666661346656454166646255541615366661366611665164261456426656266652313356652641251526625653326632666656666666666343636663152662666644553265631664646466666256613456614536256661354633564653632525364616312644364446164646156256526552516

Matricu $V$ računamo rekurzivno prema formuli:
$$V_k(i)=e_k(i)\max_{j\in\{0,1\}} \left(a_{jk}V_j(i-1)\right)\to \boxed{\ln V_k(i)=\ln e_k(i)+ \max_{j\in\{0,1\}} \left(\ln a_{jk}+ \ln V_j(i-1)\right)}$$
gdje je $j,k\in\{0,1\}$, $a_{00}=cl$, $a_{01}=dl$, $a_{10}=df$ i $a_{11}=cf$, dok je $i$ broj koji smo dobili u tom bacanju.

Treba paziti da ne logaritmiramo više puta.

In [26]:
x = new_x

#retci emisijske matrice
F = np.log10([ 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0, 1.0/6.0 ]) #FAIR
L = np.log10([ 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/10.0, 1.0/2.0]) #LOADED

#elementi tranzicijske matrice
cl = np.log10(0.9)
dl = np.log10(0.1)
cf = np.log10(0.8)
df = np.log10(0.2)


#REKURZIVNO GENERIRANJE MATRICE V

tmp = []
V = []
for i in range(len(x)):
    tmp.append(0) #array koji sadrži nule, iste duljine kao i x

for i in range(2):
    V.append(tmp[:]) #prazna matrica 2 puta duljina od x


k = int(x[0]) - 1 
#indeks koji mi je potreban za dohvaćanje vjerojatnosti u emisijskoj matrici


V[0][0] = F[k] #vjerojatnost da je pao prvi broj (iz generiranog niza) ako sam u fair kockici
V[1][0] = L[k] #vjerojatnost da je pao prvi broj (iz generiranog niza) ako sam u floaded kockici


#rekurzivno računamo V pomoću gore navedene formule
for i in range(1,  len(x)):
    tmp = []                         #privremena lista
    k = int(x[i]) - 1                #dohvaćam indeks broja koji je pao u i-tom bacanju
    tmp.append(V[0][i-1]+F[k]+cl)    #appendam privremenu listu: log-vjer. ako smo u prošlom bili u F
                                                                # + log-vjer. da je pao broj (k+1) u F kocki
                                                                # + log-vjer. da cu ostat u L kocki?
    tmp.append(V[1][i-1]+F[k]+df)    #appendam privremenu listu: log-vjer. ako smo u prošlom bili u L
                                                                # + log-vjer. da je pao broj (k+1) u F kocki
                                                                # + log-vjer. da cu preći iz F u L?
    tmp.sort()                       #sortiram privremenu listu
    V[0][i] = tmp[1]                 #biram prvi element (maksimalni) od ta dva
    
    #ponavljam postupak od gore
    tmp = []
    tmp.append(V[1][i-1]+L[k]+cf)
    tmp.append(V[0][i-1]+L[k]+dl)
    tmp.sort()
    V[1][i] = tmp[1]    


    
#TRACEBACK
    
    
#ako mi je zadnja log-vjer. u matrici V veća u prvom redu (fair), postavi zastavicu na nula, inače 1
if V[0][len(V[0])-1] > V[1][len(V[1])-1]:
    l = 0
else:
    l = 1


le = len(V[0]) - 1    #duljina "preostalog tracebacka"
put = ''              #spremanje optimalnog puta

while le > 0 or le == 0:
    k = int(x[le])-1              #dohvaćam indeks(potrebno za emisijsku matricu)
    
    
    if le == 0:                   #slučaj kada dođemo do zadnjeg elementa
        if l == 1:                #bili smo u loaded kockici
            put = 'L' + put  
            le = le - 1
        elif l == 0:              #inače smo bili u fair kockici
            put = 'F' + put 
            le = le - 1
          
        
    elif l == 0:                  #s ovim cudnim nejednakostima provjeravam jednakost? Greška je zaokruživanje?
                                  #sigurno ću dodati F u put jer l=0
                                  #provjeravam hoću li ostati u F ili prelazim u L (traceback po formuli)
        if (( V[l][le]/( V[l+1][le-1]+(F[k]+df) ) ) > 0.99999999999 and ( V[l][le]/( V[l+1][le-1]+(F[k]+df) ) )<=1.0000000000000003 ):
            put = 'F' + put 
            le = le - 1
            l = l + 1
                                  #po formuli zbog df, znači da je ovo stanje proizašlo iz loaded kockice (l postaje 1)
                                  #iz L u F
                                  #analogno slijedi za ostale slučajeve
        elif ( ( V[l][le]/( V[l][le-1]+(F[k]+cl) ) ) > 0.9999999999 and ( V[l][le]/( V[l][le-1]+(F[k]+cl) ) )<=1.0000000000000003 ):
            put = 'F' + put
            le = le - 1
     
    
    
    elif l == 1:
        if ( ( V[l][le]/( V[l-1][le-1]+(L[k]+dl) ) ) > 0.99999999999 and ( V[l][le]/( V[l-1][le-1]+(L[k]+dl) ) )<=1.0000000000000003 ):
            put = 'L' + put  
            le = le - 1
            l = l - 1
        elif ( ( V[l][le]/( V[l][le-1]+(L[k]+cf) ) ) > 0.999999999 and ( V[l][le]/( V[l][le-1]+(L[k]+cf) ) )<=1.0000000000000003 ):
            put = 'L' + put
            le = le - 1        
print(put)   

1000
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLFFFFFFFFFFFFFFFLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF

Zadatak 2: u narednim zadacima cemo se baviti optimizacijom modela, bolje receno, optimizacijom parametara modela; ako imamo kockarnicu sa samo jednom kockom i dani niz - oznacimo ga sa D (data) - onda je jasno koji su parametri θ
θkoji maksimiziraju vjerodostojnost podataka, tj.

(dokazite, tj. pronadjite dokaz); no, ako se nas model sastoji od barem dvije kocke, ne postoji zatvorena formula kojom bismo odredili optimalne parametre; zbog toga se sluzimo raznim aproksimacijama i/ili iterativnim procedurama da "pronadjemo" optimalne parametre; u ovom cemo zadatku napraviti dio koraka za iterativnu/aproksimativnu optimizaciju parametara modela (pomocu Viterbijevog algoritma); dakle, zadatak je:

i) simulirati dugacak niz (5000 ili 10000 znakova, recimo) iz nekog modela (s dvije kocke)

ii) za neki model, s nekim drugim parametrima (razlicitim, ali ne jako razlicitim), pronaci optimalan put/prolaz niza kroz model

iii) za tako dobiveni prolaz, odrediti emisijske i tranzicijske frekvencije (npr. kocka 1 je znak 1 emitirala 891 put, znak 2 902 puta, itd...., analogno za tranzicije); nakon normalizacije (tako da se odgovarajuci parametri sumiraju na 1), imate "nove" parametre modela

iv) provjerite koliko su vam tako dobiveni "novi" parametri slicni pocetnima (onima iz kojih ste simulirali)